In [1]:
import sys, os
sys.path.insert(0, "/home/gstupp/projects/wikidata-biothings/scheduled_bots")
sys.path.insert(0, "/home/gstupp/projects/WikidataIntegrator/")

In [14]:
import json
from collections import defaultdict
import pytablewriter
import pandas as pd
pd.options.display.max_colwidth = 0
from collections import OrderedDict
from scheduled_bots.ontology.DOID_obographs_bot import *
json_path='doid.json'

In [3]:
with open(json_path) as f:
    d = json.load(f)
graphs = {g['id']: g for g in d['graphs']}
graph = graphs['http://purl.obolibrary.org/obo/doid.owl']
do = DOGraph(graph, None, True)

In [4]:
d = defaultdict(set)
for node in do.nodes.values():
    for xref in node.xrefs:
        d[xref].add(node.id)

In [5]:
dupes = sorted({k:v for k,v in d.items() if len(v)>1}.items(), key=lambda x:len(x[1]), reverse=True)
len(dupes)

1496

In [6]:
dupe_key = defaultdict(dict)
for k,v in dupes:
    prefix, _id = k.split(":", 1)
    dupe_key[prefix][_id] = v

In [7]:
{k:len(v) for k,v in dupe_key.items()}

{'CSP': 1,
 'ICD10CM': 280,
 'ICD9CM': 35,
 'ICDCM10': 1,
 'KEGG': 2,
 'MESH': 200,
 'NCI': 37,
 'OMIM': 46,
 'ORDO': 40,
 'SNOMEDCT_US_2016_03_01': 803,
 'UMLS_CUI': 51}

In [8]:
dupe_key['ORDO']

{'101046': {'http://purl.obolibrary.org/obo/DOID_0060748',
  'http://purl.obolibrary.org/obo/DOID_0060751',
  'http://purl.obolibrary.org/obo/DOID_0060754'},
 '103908': {'http://purl.obolibrary.org/obo/DOID_0060777',
  'http://purl.obolibrary.org/obo/DOID_0060781'},
 '137783': {'http://purl.obolibrary.org/obo/DOID_0060560',
  'http://purl.obolibrary.org/obo/DOID_0060653'},
 '140957': {'http://purl.obolibrary.org/obo/DOID_0060691',
  'http://purl.obolibrary.org/obo/DOID_0090102'},
 '1496': {'http://purl.obolibrary.org/obo/DOID_0060600',
  'http://purl.obolibrary.org/obo/DOID_0090003'},
 '163717': {'http://purl.obolibrary.org/obo/DOID_0060749',
  'http://purl.obolibrary.org/obo/DOID_0060750',
  'http://purl.obolibrary.org/obo/DOID_0060752'},
 '169147': {'http://purl.obolibrary.org/obo/DOID_0060295',
  'http://purl.obolibrary.org/obo/DOID_0060297',
  'http://purl.obolibrary.org/obo/DOID_0060298'},
 '169150': {'http://purl.obolibrary.org/obo/DOID_0060299',
  'http://purl.obolibrary.org/obo

In [23]:
ordo_url = "http://www.orpha.net/consor/cgi-bin/OC_Exp.php?lng=EN&Expert="
df = pd.DataFrame([{'notes': '', 'ordo': ordo_url + k, "do": "\n".join("{} ({})".format(do.nodes[x].lbl,x) if x in do.nodes else "*" for x in v )} for k,v in dupe_key['ORDO'].items()])
df = df[['ordo', 'do', 'notes']]

In [24]:
writer = pytablewriter.MarkdownTableWriter()
writer.header_list = list(df.columns.values)
writer.value_matrix = df.values.tolist()
writer.write_table()

|                               ordo                                |                                                                                                                                                                                                                                                                           do                                                                                                                                                                                                                                                                           |notes|
|-------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------